In [1]:
from rdflib import Graph
import pprint

In [2]:
g = Graph()
g.parse("outputs/output.txt", format="turtle")

<Graph identifier=Ne848499b7c3d440da2364be46a801766 (<class 'rdflib.graph.Graph'>)>

In [3]:
qres = g.query(
    """
    PREFIX xsd:  <http://www.w3.org/2001/XMLSchema#>
    PREFIX geo:  <http://www.geonames.org/ontology#>
    PREFIX my:   <http://my.uri/ontology/>
    SELECT ?a ?b ?c
       WHERE {
          ?_ geo:name "Germany" ; 
           geo:parentFeature ?b .
          ?c geo:parentFeature ?b .
          ?c geo:alternateName ?a .
          filter(lang(?a) = "el")
       }""")

print("{: <25} {: <35}   {: <35}".format("Country Name", "Parent Feature ID", "Country ID"))
for row in qres:
    print("{: <25} {: <35}   {: <35}".format(row[0], row[1], row[2]))

Country Name              Parent Feature ID                     Country ID                         
Αυστρία                   https://sws.geonames.org/12217848/    https://sws.geonames.org/2782113/  
Σλοβενία                  https://sws.geonames.org/12217848/    https://sws.geonames.org/3190538/  
Πολωνία                   https://sws.geonames.org/12217848/    https://sws.geonames.org/798544/   
Γερμανία                  https://sws.geonames.org/12217848/    https://sws.geonames.org/2921044/  
Σλοβακία                  https://sws.geonames.org/12217848/    https://sws.geonames.org/3057568/  
Τσεχία                    https://sws.geonames.org/12217848/    https://sws.geonames.org/3077311/  
Ουγγαρία                  https://sws.geonames.org/12217848/    https://sws.geonames.org/719819/   
Βέλγιο                    https://sws.geonames.org/9408659/     https://sws.geonames.org/2802361/  
Γερμανία                  https://sws.geonames.org/9408659/     https://sws.geonames.org/2921044/  


In [4]:
qres = g.query(   
    """
    PREFIX xsd:  <http://www.w3.org/2001/XMLSchema#>
    PREFIX geo:  <http://www.geonames.org/ontology#>
    PREFIX my:   <http://my.uri/ontology/>
    SELECT ?a ?pop ?c
       WHERE {
          ?d geo:name ?a ;
             geo:parentFeature <https://sws.geonames.org/12217848/> ;
             my:population ?pop ;
             my:may_total_vaccinations ?c .
       }""")


print("{: <25} {: <15}  {: <15}".format("Country Name", "Population", "Total May Vaccinations %"))
for row in qres:
    print("{: <25} {: <15}  {: <15}".format(row[0], row[1], "{:.2f}".format(float(row[2]))))

Country Name              Population       Total May Vaccinations %
Austria                   9015361          38.36          
Slovenia                  2078989          33.80          
Poland                    37839255         34.45          
Germany                   83830972         40.04          
Slovakia                  5460109          31.88          
Czechia                   10712481         32.76          
Hungary                   9655983          66.99          
Switzerland               8665615          32.22          


In [5]:
qres = g.query(   
    """
    PREFIX xsd:  <http://www.w3.org/2001/XMLSchema#>
    PREFIX geo:  <http://www.geonames.org/ontology#>
    PREFIX my:   <http://my.uri/ontology/>
    SELECT DISTINCT ?b (SUM(?res) AS ?total_people) (SUM(?pop) as ?population)
       WHERE {
          ?_ geo:name "Greece" ; 
             geo:parentFeature ?b .
          ?d geo:parentFeature ?b ;
             my:may_total_vaccinations ?c ;
             my:population ?pop .
          BIND((?c*?pop/100) AS ?res)
       }
       GROUP BY ?b""")

print("{: <40} {: <25}  {: <15}".format("Parent Feature ID", "Total Vaccinations", "Total Population"))
for row in qres:
    print("{: <40} {: <25}  {: <15}".format(row[0], "{:.0f}".format(float(row[1])), row[2]))

Parent Feature ID                        Total Vaccinations         Total Population
https://sws.geonames.org/9408658/        54636706                   148122497      
https://sws.geonames.org/6695072/        164176267                  445294078      
https://sws.geonames.org/12216908/       16112061                   60471243       
https://sws.geonames.org/12217088/       116373164                  489091807      


In [6]:
qres = g.query(   
    """
    PREFIX xsd:  <http://www.w3.org/2001/XMLSchema#>
    PREFIX geo:  <http://www.geonames.org/ontology#>
    PREFIX my:   <http://my.uri/ontology/>
    SELECT ?a
       WHERE {
          ?b my:Johnson 1 .
          ?b geo:name ?a
       }""")

for row in qres:
    print(row[0])

Lithuania
South Africa
Belgium
Czechia
Italy
Latvia
Spain
Netherlands
Germany
Poland
Austria
Romania
France


In [7]:
qres = g.query(
    """
    PREFIX xsd:  <http://www.w3.org/2001/XMLSchema#>
    PREFIX geo:  <http://www.geonames.org/ontology#>
    PREFIX my:   <http://my.uri/ontology/>
    SELECT ?a ?c
       WHERE {
          ?count geo:name ?a .
          ?count my:NY.GDP.PCAP.CD ?c .
          MINUS{?count geo:parentFeature <https://sws.geonames.org/6695072/>}
            
       }""")

print("{: <25}  {: <35}".format("Country Name",  "Country ID"))
for row in qres:
    print("{: <25}  {: <35}".format(row[0], row[1]))

Country Name               Country ID                         
Ukraine                    2471.752                           
Mexico                     8646.151                           
Nicaragua                  1498.502                           
Argentina                  8731.126                           
Australia                  44709.85                           
Honduras                   1740.344                           
Nepal                      492.9654                           
Kazakhstan                 7351.106                           
Costa Rica                 7389.976                           
Canada                     41053.51                           
Armenia                    2690.334                           
Djibouti                   1210.808                           
Azerbaijan                 4218.524                           
Zambia                     1129.012                           
Philippines                1909.211                    

Lets make a scenario. We want the total may vaccinations from the countries that have a GDP larger than the average European Union GDP (and don't belong in EU).
First of all we want to calculate the average GDP of the countries in the EU. EU's Feuature ID is https://sws.geonames.org/6695072/, so we will use this to filter the query. We estimate the vaccination % by dividing the doses % by 2.

In [30]:
qres = g.query(
    """
    PREFIX xsd:  <http://www.w3.org/2001/XMLSchema#>
    PREFIX geo:  <http://www.geonames.org/ontology#>
    PREFIX my:   <http://my.uri/ontology/>
    SELECT (COUNT(?a) as ?cnt) (AVG(?c) as ?avg)
       WHERE {
          ?count geo:name ?a .
          ?count my:NY.GDP.PCAP.CD ?c .
          ?count geo:parentFeature <https://sws.geonames.org/6695072/>       
       }""")

print("{: <25}  {: <35}".format("Number of Countries",  "Average GDP"))
for row in qres:
    print("{: <25}  {: <35}".format(row[0], "{:.2f}".format(float(row[1]))))

Number of Countries        Average GDP                        
27                         28666.84                           


Next let's find the countries not in the EU, with their respective GDPs

In [31]:
qres = g.query(
    """
    PREFIX xsd:  <http://www.w3.org/2001/XMLSchema#>
    PREFIX geo:  <http://www.geonames.org/ontology#>
    PREFIX my:   <http://my.uri/ontology/>
    SELECT ?a ?c
       WHERE {
          ?count geo:name ?a .
          ?count my:NY.GDP.PCAP.CD ?c .
          MINUS{?count geo:parentFeature <https://sws.geonames.org/6695072/>}       
       }""")

print("{: <25}  {: <35}".format("Countries",  "GDP"))
for row in qres:
    print("{: <25}  {: <35}".format(row[0], "{:.2f}".format(float(row[1]))))

Countries                  GDP                                
Ukraine                    2471.75                            
Mexico                     8646.15                            
Nicaragua                  1498.50                            
Argentina                  8731.13                            
Australia                  44709.85                           
Honduras                   1740.34                            
Nepal                      492.97                             
Kazakhstan                 7351.11                            
Costa Rica                 7389.98                            
Canada                     41053.51                           
Armenia                    2690.33                            
Djibouti                   1210.81                            
Azerbaijan                 4218.52                            
Zambia                     1129.01                            
Philippines                1909.21                     

Combining the queries above, we can now calculate the average May total vaccinations of EU countries vs non EU countries with higher GDP than the average EU country

In [33]:
qres = g.query(
    """
    PREFIX xsd:  <http://www.w3.org/2001/XMLSchema#>
    PREFIX geo:  <http://www.geonames.org/ontology#>
    PREFIX my:   <http://my.uri/ontology/>
    SELECT (COUNT(?a) as ?cnt) (AVG(?res) as ?avg)
       WHERE {
          ?count geo:name ?a .
          ?count my:NY.GDP.PCAP.CD ?c ;
                 my:may_total_vaccinations ?vac .
          MINUS{?count geo:parentFeature <https://sws.geonames.org/6695072/>}
          FILTER(?c > 28666.8)
          BIND((?vac / 2) AS ?res)
       }""")


print("Non EU countries with higher than average EU-GDP\n")
print("{: <25}  {: <35}".format("Number of Countries",  "Average May vacc %"))
for row in qres:
    print("{: <25}  {: <35}".format(row[0], "{:.2f}".format(float(row[1]))))

Non EU countries with higher than average EU-GDP

Number of Countries        Average May vacc %                 
9                          17.19                              


In [34]:
qres = g.query(
    """
    PREFIX xsd:  <http://www.w3.org/2001/XMLSchema#>
    PREFIX geo:  <http://www.geonames.org/ontology#>
    PREFIX my:   <http://my.uri/ontology/>
    SELECT (COUNT(?a) as ?cnt) (AVG(?res) as ?avg)
       WHERE {
          ?count geo:name ?a .
          ?count my:NY.GDP.PCAP.CD ?c ;
                 my:may_total_vaccinations ?vac .
          MINUS{?count geo:parentFeature <https://sws.geonames.org/6695072/>}
          FILTER(?c < 28666.8)
          BIND((?vac / 2) AS ?res)
       }""")


print("Non EU countries with lower than average EU-GDP\n")
print("{: <25}  {: <35}".format("Number of Countries",  "Average May vacc %"))
for row in qres:
    print("{: <25}  {: <35}".format(row[0], "{:.2f}".format(float(row[1]))))

Non EU countries with lower than average EU-GDP

Number of Countries        Average May vacc %                 
78                         5.09                               


In [36]:
qres = g.query(
    """
    PREFIX xsd:  <http://www.w3.org/2001/XMLSchema#>
    PREFIX geo:  <http://www.geonames.org/ontology#>
    PREFIX my:   <http://my.uri/ontology/>
    SELECT (COUNT(?a) as ?cnt) (AVG(?res) as ?avg)
       WHERE {
          ?count geo:name ?a .
          ?count my:NY.GDP.PCAP.CD ?c ;
                 my:may_total_vaccinations ?vac ;
                 geo:parentFeature <https://sws.geonames.org/6695072/>
          BIND((?vac / 2) AS ?res)
       }""")

print("EU countries\n")
print("{: <25}  {: <35}".format("Number of Countries",  "Average May vacc %"))
for row in qres:
    print("{: <25}  {: <35}".format(row[0], "{:.2f}".format(float(row[1]))))

EU countries

Number of Countries        Average May vacc %                 
27                         18.08                              


In [37]:
qres = g.query(
    """
    PREFIX xsd:  <http://www.w3.org/2001/XMLSchema#>
    PREFIX geo:  <http://www.geonames.org/ontology#>
    PREFIX my:   <http://my.uri/ontology/>
    SELECT (COUNT(?a) as ?cnt) (AVG(?res) as ?avg)
       WHERE {
          ?count geo:name ?a .
          ?count my:NY.GDP.PCAP.CD ?c ;
                 my:may_total_vaccinations ?vac ;
                 geo:parentFeature <https://sws.geonames.org/6695072/>
          FILTER(?c > 28666.8)
          BIND((?vac / 2) AS ?res)
       }""")


print("EU countries with higher than average EU-GDP\n")
print("{: <25}  {: <35}".format("Number of Countries",  "Average May vacc %"))
for row in qres:
    print("{: <25}  {: <35}".format(row[0], "{:.2f}".format(float(row[1]))))

EU countries with higher than average EU-GDP

Number of Countries        Average May vacc %                 
11                         18.12                              


Now we can find which countries of those 11 have higher than EU-average vaccination %

In [38]:
qres = g.query(
    """
    PREFIX xsd:  <http://www.w3.org/2001/XMLSchema#>
    PREFIX geo:  <http://www.geonames.org/ontology#>
    PREFIX my:   <http://my.uri/ontology/>
    SELECT ?a ?res
       WHERE {
          ?count geo:name ?a .
          ?count my:NY.GDP.PCAP.CD ?c ;
                 my:may_total_vaccinations ?vac ;
          MINUS{?count geo:parentFeature <https://sws.geonames.org/6695072/>}
          FILTER(?c > 28666.8)
          BIND((?vac / 2) AS ?res)
          FILTER(?res > 18.07)
       }""")


print("Non EU countries with higher than average EU-GDP and EU-Vacc%\n")
print("{: <25}  {: <35}".format("Number of Countries",  "Average May vacc %"))
for row in qres:
    print("{: <25}  {: <35}".format(row[0], "{:.2f}".format(float(row[1]))))

Non EU countries with higher than average EU-Vacc%

Number of Countries        Average May vacc %                 
Canada                     19.74                              
United Kingdom             37.69                              
Iceland                    24.47                              
Qatar                      29.98                              


In [40]:
qres = g.query(
    """
    PREFIX xsd:  <http://www.w3.org/2001/XMLSchema#>
    PREFIX geo:  <http://www.geonames.org/ontology#>
    PREFIX my:   <http://my.uri/ontology/>
    SELECT ?a ?res
       WHERE {
          ?count geo:name ?a .
          ?count my:NY.GDP.PCAP.CD ?c ;
                 my:may_total_vaccinations ?vac ;
          MINUS{?count geo:parentFeature <https://sws.geonames.org/6695072/>}
          FILTER(?c < 28666.8)
          BIND((?vac / 2) AS ?res)
          FILTER(?res > 18.07)
       }""")


print("Non EU countries with lower than average EU-GDP and higher EU-Vacc%\n")
print("{: <25}  {: <35}".format("Number of Countries",  "Average May vacc %"))
for row in qres:
    print("{: <25}  {: <35}".format(row[0], "{:.2f}".format(float(row[1]))))

Non EU countries with lower than average EU-GDP and higher EU-Vacc%

Number of Countries        Average May vacc %                 
Chile                      39.95                              
Israel                     60.43                              
Serbia                     21.28                              
Bahrain                    39.00                              
Uruguay                    28.40                              
